In [3]:
import csv
import random

import nltk
import pandas as pd
import numpy as np
import scipy.stats

In [5]:
#data prep

#read in csv, make dict with cats
#structure: 0 country | 1 date | 2 sect | 3 subsect | 4 text

rd = csv.reader(open('/Users/ewanog/Dropbox (Old)/DEEP dropbox/Assessment reports/GEO Data/2017-04-02-country-content-60-weeks.csv'))

cats = {}

for row in rd:
    sect = row[2]
    txt = row[4]
    if sect not in cats:
        cats[sect] = [txt]
    else:
        cats[sect].append(txt)
        
#counts
c=0
for k,v in cats.items():
    print(k + ' ' + str(len(v)))
    c+=len(v)
print('total count: ' + str(c))
    
print('all: ' + str(len(cats.values())))
    
#TODO: clean bad chars

#remove stopwords. STOP!!!!!!!!!ADSAAA!!W

#lower - then remove all the other calls

Education 2665
WASH 4383
Protection 8820
Politics and security 7152
Humanitarian access 4854
Displacement 7384
Shelter and NFIs 2886
Health 6255
Food security and livelihoods 9036
Nutrition 2358
total count: 55793
all: 10


In [8]:
cats['Education'][1:100]

['150,000 education staff have stopped work due to\xa0conflict, negatively impacting the entire education system in Syria. \xa0\n',
 '1.75 million children are\xa0out of school, mainly due to the violence, insecurity, and children taking up work.\xa0\n\nOver 6,000 schools -\xa0one out of every three schools -\xa0has been damaged, destroyed, occupied by militants, or used as shelter. \xa0\xa0Only 50% of schools are functional; Ar-Raqqa, Aleppo, and\xa0Deir-ez-Zor governorates have\xa0the lowest percentage\xa0of functional schools.\n\nCritical shortages of learning materials and safe learning spaces are reported in besieged and hard-to-reach areas. In host communities, classes are overcrowded.\n\nSchools in besieged and hard-to-reach areas have been operating underground.\xa0However, the reported use of "bunker busting" missiles has discouraged many children from attending underground schools.\xa0Between 18 and 28 November 2016, six schools were hit by airstrikes in Idlib governorate.\n\

In [245]:
#show duplicates
from pprint import pprint

c=0
for k,v in cats.items():
    print(k + ' ' + str(len((set(v)))))
    c+=len((set(v)))
           
print(c)

Displacement 124
Humanitarian access 80
Shelter and NFIs 48
WASH 75
Health 107
Protection 151
Education 47
Politics and security 119
Food security and livelihoods 154
Nutrition 41
946


In [157]:
#create random sampling groups
#TODO: should we do uniform random sampling per group or a whole scale random draw?

#return dicts with {sect: setted list of entries}
def gen_tt(corpus):
    test = {}
    train = {}

    #add corpus to test and train
    for k in corpus.keys():
        test[k] = None
        train[k] = None

    for k,v in corpus.items():
        random.shuffle(v)

        setv = list(set(v))
        len_test = int(len(setv)*.3)

        test[k] = setv[:len_test]
        train[k] = setv[len_test:]

    return train, test

train, test = gen_tt(cats)
#     #check lengths
#     for k in train.keys():
#         print(k + ' train : ' + str(len(train[k])))
#         print(k + ' test : ' + str(len(test[k])))

In [65]:
#test sampling (if we do 70:30)

d = {
    'cat1': [random.randint(0,100) for i in range(100)],
    'cat2': [random.randint(0,100) for i in range(150)]
}

test = {}
train = {}

#add cats to test and train
for k in d.keys():
    test[k] = None
    train[k] = None

for k,v in d.items():
    len_test = int(len(v)*.3)
    print(len(v)*.3)
    
    test[k] = v[:len_test]
    train[k] = v[len_test:]
    
print(len(test))
print(len(train))

45.0
30.0
2
2


In [225]:
#start building our model
#train and test are just subsets of cats (for now)

#make a feature with word presence
#TODO: what's python func?
all_words = []
for k,v in cats.items():
    for e in list(set(v)):
        for w in e.split(' '):
            all_words.append(w.lower())

freq_words = nltk.FreqDist(all_words)

def document_features(document):
#     if type(document) == list:
#         #TODO: func
#         grp = ''
#         for l in document:
#             for w in l:
#                 grp += w
#         document = grp
                
    uniq_doc = set(document.split(' '))
    features = {}
    for word in list(freq_words.keys())[:2000]:
        features['contains(%s)' % word] = (word in uniq_doc)
    return features

#convert dicts into list of tuples we need
#TODO: func
train_merge = []
test_merge = []

for k,v in train.items():
    for i in v:
        train_merge.append((k,i))

for k,v in test.items():
    for i in v:
        test_merge.append((k,i))

        
print(len(test_merge))

wp_train = [(document_features(v), k) for k,v in train_merge]
wp_test = [(document_features(v), k) for k,v in test_merge]

nb_class = nltk.NaiveBayesClassifier.train(wp_train)

# wp_test[0]

281


In [229]:
#results

#from just raw, unrefined features and 200 words: 0.398576512455516

nb_class.show_most_informative_features(100)

Most Informative Features
       contains(primary) = True           Educat : Food s =     35.3 : 1.0
     contains(political) = True           Politi : Food s =     25.2 : 1.0
        contains(houses) = True           Shelte : WASH   =     15.9 : 1.0
      contains(outbreak) = True           Health : Politi =     14.5 : 1.0
        contains(asylum) = True           Displa : Food s =     14.5 : 1.0
    contains(registered) = True           Displa : Food s =     14.1 : 1.0
         contains(until) = True           Politi : Protec =     13.0 : 1.0
         contains(women) = True           Protec : Politi =     13.0 : 1.0
          contains(army) = True           Politi : Food s =     11.5 : 1.0
           contains(but) = True           Politi : Shelte =     11.1 : 1.0
            contains(17) = True           Politi : Food s =     10.7 : 1.0
      contains(fighters) = True           Politi : Displa =     10.0 : 1.0
         contains(rains) = True           Shelte : Health =      9.4 : 1.0

In [224]:
# ['contains(hit)',
#  'contains(down)',
#  'contains(being)',
#  'contains(member.)',
#  'contains(17)',
#  'contains(other)',
#  'contains(air)',
#  'contains(redirect,)',
#  'contains(released)',
#  'contains(independence)',
#  'contains(until)',
#  'contains(the)']

# [k for k,v in wp_train[100][0].items() if v is True]

wp_train[100][1]

'Humanitarian access'

In [185]:
for k,v in train.items():
    print(k)
    print(v)
    break

Displacement
["By 31 January, there were an estimated 1.9\xa0million IDPs across six states\xa0in northeast Nigeria,\xa0an increase of 7% from the December assessment. The overall figure includes 1.5\xa0million\xa0in\xa0Borno,\xa0147,000\xa0in Adamawa, and 113,000 in Yobe. 29.5%\xa0of these IDPs have been displaced in 2016. 97% have been displaced due to the\xa0BH\xa0insurgency, and 3% due to communal clashes. 86.8% of IDPs originate from Borno, 5.7% from Adamawa, and 4.2% from Yobe.\xa0\xa0By the end of February it was reported that IDPs were increasingly moving from Maiduguri to newly accessible LGAs.\xa0\xa0This includes the 100 families that were returning per day to Damasak LGA in February.\xa0\n\nCounter insurgency operations in Chibok, Damboa, Gwoza, Kala/Balge, Mafa and Monguno LGAs in Borno state Between 25 February and 1 March, at least 7,000 people were displaced from Chibok in Borno state due to renewed BH attacks. \xa0Between 27 January and 10 February, military operations

In [155]:
wpfeatures[0][0]

{'contains(mechanism.\xa0the)': False,
 'contains(modira,)': False,
 'contains(report)': True,
 'contains(cadre)': False,
 'contains(dineo,\xa0including)': False,
 'contains(schoolgirls\xa0remain)': False,
 'contains(buenaventura)': False,
 'contains(337\xa0deaths)': False,
 'contains(25,000\xa0in)': True,
 'contains(camp.\xa0at)': False,
 'contains(critically)': False,
 'contains(by\xa0security)': False,
 'contains(to\xa0one)': False,
 'contains(strengthened)': True,
 'contains(metals)': False,
 'contains(yet.)': True,
 'contains(acknowledge)': True,
 'contains(euros)': False,
 'contains(idps))': False,
 'contains(tensions)': True,
 'contains(curbs)': False,
 'contains(area.a)': False,
 'contains(tributaries,)': False,
 'contains(wounded.the)': False,
 'contains(refrain)': False,
 'contains(territory.)': True,
 'contains(public)': True,
 'contains(darfur,\xa0have)': False,
 'contains(issoufou)': False,
 'contains(expenditure.)': False,
 'contains(bathroom.)': False,
 'contains(saw\xa0